### Find Word Group Youtube Link

In [25]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [26]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [27]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 40000  # 28 native word end index

# youtube
sample_num = 10  # 7
time_shift = 0.6

In [28]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False)].sample(sample_num)
            #df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].tail(sample_num)  # will test
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [29]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")\n
    df_search_result is dataframe and "search_string", "start_time", "end_time", "sentence", "video_id" are its columns
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [30]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [31]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [32]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 6/Result/2-Find Word Group Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

In [33]:
## Master File
#path_folder_file = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 5/Data/Deployment/Talk Time 5 Master File.xlsx"
#sheet = "Sheet6"  # Sheet1
#file_ext = "Sixgram"

In [121]:
ngram = "Twogram"
path_folder_file = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 6/\
Result/1-Select Twogram Threegram According To Word Usage Limit/{lang_folder.capitalize()}_{ngram}_Selected_With_{word_end}_Word.xlsx"
sheet = "Sheet1"
file_ext = ngram

In [122]:
df_file = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
#df_file = pd.read_excel(f"twogram.xlsx")
df_file

,twogram,frequency
0,burada ne,93996
1,sana bir,93343
2,kapa çeneni,79080
3,sen ne,72768
4,de öyle,68645
...,...,...
194,rica ederim,21163
195,benim hatam,21141
196,çok özür,21109
197,sen misin,21097


In [123]:
search_list = df_file.iloc[:,0].to_list()
len(search_list)

199

In [124]:
#disable_video_id_list = ["H6E6N70jYqI","BoaYsdPtJYA","HUZINsU40Fk","Et3diPcEmfY"]
disable_video_id_list = []

In [125]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [126]:
df_youtube_sentence = df_youtube_sentence[~df_youtube_sentence["video_id"].isin(disable_video_id_list)]
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [127]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [128]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [129]:
df_word_group = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
df_word_group

,search_string,start_time,end_time,sentence,video_id
0,burada ne,1799.721,1801.971,bu manken kılıklının burada ne işi var,AxEi0Uy8gMQ
1,burada ne,490.102,490.933,benim burada ne işim var,ZjeNTjm_uo0
2,burada ne,972.298,973.623,bu kızın burada ne işi var,xEx954Qsjjc
3,burada ne,1970.769,1973.174,asıl sen burada ne arıyorsun ha,HeXezF4VrJ8
4,burada ne,254.590,261.250,açalım isterseniz hemen şuradan bindik arka ka...,FGUTSQb5dPM
...,...,...,...,...,...
1973,bekle bir,318.680,328.670,bekle diyor size enerji sevgili arkadaşlar bek...,3rtbPiaDIDo
1974,bekle bir,8813.173,8814.316,bekle bir saniye,zLSi0My6vuQ
1975,bekle bir,204.032,206.080,firmada elifa bekle bir sorun yaşadığını görüy...,8tm4tpBBAdM
1976,bekle bir,5713.833,5716.500,tamam tamam ben senin kızın değilim bekle bir ...,aPwTtMXPv3o


In [130]:
df_word_group_time_loc = word_group_time_loc(df_word_group, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,burada ne,1800.905211,1801.556526,bu manken kılıklının burada ne işi var,AxEi0Uy8gMQ
1,burada ne,490.275125,490.656000,benim burada ne işim var,ZjeNTjm_uo0
2,burada ne,972.705692,973.266269,bu kızın burada ne işi var,xEx954Qsjjc
3,burada ne,1971.389645,1972.243032,asıl sen burada ne arıyorsun ha,HeXezF4VrJ8
4,burada ne,260.518132,261.250000,açalım isterseniz hemen şuradan bindik arka ka...,FGUTSQb5dPM
...,...,...,...,...,...
1973,bekle bir,325.515263,326.672000,bekle diyor size enerji sevgili arkadaşlar bek...,3rtbPiaDIDo
1974,bekle bir,8813.173000,8813.887375,bekle bir saniye,zLSi0My6vuQ
1975,bekle bir,204.564480,205.015040,firmada elifa bekle bir sorun yaşadığını görüy...,8tm4tpBBAdM
1976,bekle bir,5715.388750,5715.877700,tamam tamam ben senin kızın değilim bekle bir ...,aPwTtMXPv3o


In [131]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,burada ne,1800.305211,1802.156526,bu manken kılıklının burada ne işi var,AxEi0Uy8gMQ
1,burada ne,489.675125,491.256000,benim burada ne işim var,ZjeNTjm_uo0
2,burada ne,972.105692,973.866269,bu kızın burada ne işi var,xEx954Qsjjc
3,burada ne,1970.789645,1972.843032,asıl sen burada ne arıyorsun ha,HeXezF4VrJ8
4,burada ne,259.918132,261.850000,açalım isterseniz hemen şuradan bindik arka ka...,FGUTSQb5dPM
...,...,...,...,...,...
1973,bekle bir,324.915263,327.272000,bekle diyor size enerji sevgili arkadaşlar bek...,3rtbPiaDIDo
1974,bekle bir,8812.573000,8814.487375,bekle bir saniye,zLSi0My6vuQ
1975,bekle bir,203.964480,205.615040,firmada elifa bekle bir sorun yaşadığını görüy...,8tm4tpBBAdM
1976,bekle bir,5714.788750,5716.477700,tamam tamam ben senin kızın değilim bekle bir ...,aPwTtMXPv3o


In [132]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: round(x))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: round(x))
df_word_group_time_loc 

,search_string,start_time,end_time,sentence,video_id
0,burada ne,1800,1802,bu manken kılıklının burada ne işi var,AxEi0Uy8gMQ
1,burada ne,490,491,benim burada ne işim var,ZjeNTjm_uo0
2,burada ne,972,974,bu kızın burada ne işi var,xEx954Qsjjc
3,burada ne,1971,1973,asıl sen burada ne arıyorsun ha,HeXezF4VrJ8
4,burada ne,260,262,açalım isterseniz hemen şuradan bindik arka ka...,FGUTSQb5dPM
...,...,...,...,...,...
1973,bekle bir,325,327,bekle diyor size enerji sevgili arkadaşlar bek...,3rtbPiaDIDo
1974,bekle bir,8813,8814,bekle bir saniye,zLSi0My6vuQ
1975,bekle bir,204,206,firmada elifa bekle bir sorun yaşadığını görüy...,8tm4tpBBAdM
1976,bekle bir,5715,5716,tamam tamam ben senin kızın değilim bekle bir ...,aPwTtMXPv3o


In [133]:
df_word_group_time_loc["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc['video_id'].map(str)+"&t="+df_word_group_time_loc['start_time'].map(str)+"s"
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id,video_url
0,burada ne,1800,1802,bu manken kılıklının burada ne işi var,AxEi0Uy8gMQ,https://www.youtube.com/watch?v=AxEi0Uy8gMQ&t=...
1,burada ne,490,491,benim burada ne işim var,ZjeNTjm_uo0,https://www.youtube.com/watch?v=ZjeNTjm_uo0&t=...
2,burada ne,972,974,bu kızın burada ne işi var,xEx954Qsjjc,https://www.youtube.com/watch?v=xEx954Qsjjc&t=...
3,burada ne,1971,1973,asıl sen burada ne arıyorsun ha,HeXezF4VrJ8,https://www.youtube.com/watch?v=HeXezF4VrJ8&t=...
4,burada ne,260,262,açalım isterseniz hemen şuradan bindik arka ka...,FGUTSQb5dPM,https://www.youtube.com/watch?v=FGUTSQb5dPM&t=...
...,...,...,...,...,...,...
1973,bekle bir,325,327,bekle diyor size enerji sevgili arkadaşlar bek...,3rtbPiaDIDo,https://www.youtube.com/watch?v=3rtbPiaDIDo&t=...
1974,bekle bir,8813,8814,bekle bir saniye,zLSi0My6vuQ,https://www.youtube.com/watch?v=zLSi0My6vuQ&t=...
1975,bekle bir,204,206,firmada elifa bekle bir sorun yaşadığını görüy...,8tm4tpBBAdM,https://www.youtube.com/watch?v=8tm4tpBBAdM&t=...
1976,bekle bir,5715,5716,tamam tamam ben senin kızın değilim bekle bir ...,aPwTtMXPv3o,https://www.youtube.com/watch?v=aPwTtMXPv3o&t=...


In [134]:
word_count_result(df_word_group_time_loc, ["search_string"], set_condition=True)

,word,word_count
0,bir,27
1,ne,21
2,bu,13
3,sen,11
4,ben,9
...,...,...
196,yapacaksın,1
197,yapacağım,1
198,yapmak,1
199,şaka,1


In [135]:
df_word_group_time_loc.to_excel(f"{lang_folder.capitalize()}_Word_Group_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result_{file_ext}.xlsx", index=False) 

#### Copy Move And Delete

In [136]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result_*.xlsx")
output_file

['Turkish_Word_Group_With_40000_Word_10_Youtube_0.6s_Timeshift_For_Talk_Time_Result_Twogram.xlsx']

In [137]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [138]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass